In [1]:
#compute the eigenvectors of Schur's matrix according to Morton's '78 paper, "On the Eigenvectors of Schur's Matrix"
#https://www.sciencedirect.com/science/article/pii/0022314X80900839

In [59]:
#compute the eigenvectors of Schur's matrix (symbolically) over the complex numbers (or a cyclotomic field containing q^th roots of unity)
#note: A is not unitary 
q=5
alpha = exp(2*pi*I/q)
#K.<alpha> = CyclotomicField(3)
A = matrix([[(alpha**(i*j)).expand() for j in range(1,q+1)] for i in range(1,q+1)])
A.eigenvectors_right()

[(sqrt(5),
  [(1, 0, 0, 1, 1/2*sqrt(5) + 1/2), (0, 1, 1, 0, 1/2*sqrt(5) + 1/2)],
  2),
 (-I*sqrt(5),
  [(1, -1/2*sqrt(5) - 1/2*sqrt(2*sqrt(5) + 10) - 1/2, 1/2*sqrt(5) + 1/2*sqrt(2*sqrt(5) + 10) + 1/2, -1, 0)],
  1),
 (I*sqrt(5),
  [(1, -1/2*sqrt(5) + 1/2*sqrt(2*sqrt(5) + 10) - 1/2, 1/2*sqrt(5) - 1/2*sqrt(2*sqrt(5) + 10) + 1/2, -1, 0)],
  1),
 (-sqrt(5), [(1, 1, 1, 1, -sqrt(5) + 1)], 1)]

In [27]:
#compute the eigenvectors of the DFT matrix (symbolically) over the complex numbers (or a cyclotomic field containing q^th roots of unity)
#NOTE: The DFT matrix is subtly different than the Schur matrix
#DFT = exp(2*pi*i*m*n/q) for 0 <= m,n <= q-1
#whereas for Schur's matrix A it is 1 <= m,n <= q
#this amounts to applying the cyclic permutation \sigma = (1 2 ... n) to Schur's matrix to both rows and columns
#if the row permutation is P_\sigma, the column permutation is P_\sigma^{T} = P_\sigma^{-1}
#then DFT = P_\sigma*A*P_\sigma^{-1}, so the two matrices are similar and have the same eigenvalues
#the eigenvectors of DFT are Pv if v is an eigenvector of A, i.e. cyclicly permuting the entries of v
DFT = matrix([[(alpha**(i*j)).expand() for j in range(q)] for i in range(q)])
DFT.eigenvectors_right()

[(2*I, [(0, 1, 0, -1)], 1),
 (-2, [(1, -1, -1, -1)], 1),
 (2, [(1, 0, 1, 0), (0, 1, -2, 1)], 2)]

In [28]:
#check that A is similar to the DFT matrix by cyclicly permuting columns and rows
sigma = Permutation([(i % q)+1 for i in range(1,q+1)])
P = sigma.to_matrix()
P.inverse()*DFT*P == A

True

In [158]:
#define the vectors X_d given a divisor d of q/f(chi), where f(chi) is the conductor of the character \chi
#characters are evaluated as primitive characeters, i.e. relative to the conductor
def X(d,chi):
    q = chi.modulus()
    f = chi.conductor()
    assert d.divides(q/f)
    return vector([chi.primitive_character()(n/d) if d.divides(n) else 0 for n in range(1,q+1)])

In [30]:
#helper function to embed cyclotomic field into symbolic ring
def embed_SR(x):
    K = x.parent()
    K.<z> = K
    order = K.gen().multiplicative_order()
    phi = K.hom([exp(2*pi*I/order)],SR)
    return phi(x)

In [156]:
#compute eigenvectors given a character chi and a divisor d of q/f(chi)
def eigenvectors(d,chi):
    q = chi.modulus()
    f = chi.conductor()
    try:
        assert d.divides(q/f)
    except AssertionError:
        print("Error: d must divide q/f(chi)")
        return

    #define the eigenvalues
    eig_plus = sqrt(chi(-1)*q)
    eig_minus = -sqrt(chi(-1)*q)
    eig_X = sqrt(q/f)*chi.primitive_character().gauss_sum()
    #map the eigenvalue to the symbolic ring if not already a member
    if eig_X.parent() != SR:
        eig_X = embed_SR(eig_X)
    eig_X = eig_X.simplify_full()

    #define the eigenvectors E(chi,d,lambda) as in (13) in the paper
    #need to restrict character to modulus conductor, i.e. the primitive character|
    E = lambda chi, d, eig: sqrt(d)*X(d,chi) + (eig/(chi.bar().primitive_character().gauss_sum()*sqrt(d)))*X(Integer(q/(f*d)),chi.bar())

    #check if chi is real and if the ratio of modulus to conductor is the square of a divisor
    reality_condition = chi.bar() == chi
    square_condition = d^2 == q/f

    #if chi is real and square condition is met, just return X_d(chi)
    if reality_condition and square_condition:
        return [(eig_X, X(d,chi))]

    #if one or the other condition is met, return a pair of eigenvalues corresponding to E(chi,d,\pm lambda)
    if reality_condition != square_condition:
        return (eig_plus, E(chi,d,eig_plus)), (eig_minus, E(chi,d,eig_minus))

    #if neither condition are met, return four eigenvalues E(chi,d,\pm lambda) and E(chi_bar,d,\pm lambda)
    if (not reality_condition) and (not square_condition):
        return (eig_plus,E(chi,d,eig_plus)), (eig_minus,E(chi,d,eig_minus)), (eig_plus,E(chi.bar(),d,eig_plus)), (eig_minus,E(chi.bar(),d,eig_minus))

In [114]:
from sage.misc.flatten import flatten
#compute all the eigenvectors the Schur matrix
#return a dict with eigenvalues as keys and list of eigenvectors as values
def all_eigenvectors(q):
    all_eigs = {}
    G = DirichletGroup(q)
    for chi in G:
        f = chi.conductor()
        for d in divisors(q/f):
            for v in eigenvectors(Integer(d),chi):
                if v[0] not in all_eigs:
                    all_eigs[v[0]] = [v[1]]
                else:
                    all_eigs[v[0]].append(v[1])
    return all_eigs

In [8]:
#BUG 1:
#for q=3 and q=4, it seems that the second condition is being called twice when it should only be called once
#it's giving two independent eigenvectors each time, but they are dependent on other vectors in the list
#there are two total extra vectors, so removing these two would fix the problem
#NOTE: zeta4 <--> I, zeta6 <--> exp(2*pi*I/6), etc. eigenvalues = {\pm sqrt(q), \pm I*sqrt(q)}. use map to symbolic ring.

In [126]:
#BUG 2:
#for q=5, the eigenvector component expressions are very complicated because they're in the symbolic ring
#any linear combintation is an eigenvector, so perhaps these are complicated linear combinations of the simple ones by eigenvectors_right()
#for sqrt(5), it is almost correct. the last component is sqrt(5)/2 when it should be sqrt(5)/2+1/2
#these are occuring when (reality_condition == True) and (square_condition == False)

In [150]:
chi = DirichletGroup(5)[0]; chi

Dirichlet character modulo 5 of conductor 1 mapping 2 |--> 1

In [153]:
chi(5)

0

In [157]:
all_eigenvectors(5)

{sqrt(5): [(1, 1, 1, 1, sqrt(5) + 1),
  (1, 1, 1, 1, sqrt(5) + 1),
  (1, -1, -1, 1, 0)],
 -sqrt(5): [(1, 1, 1, 1, -sqrt(5) + 1), (-1, -1, -1, -1, sqrt(5) - 1)],
 sqrt(-5): [(-sqrt(-5)/((1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^7 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^6 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^4 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^3) + 1, I*sqrt(-5)/((1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^7 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^6 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^4 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^3) + I, -I*sqrt(-5)/((1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^7 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^6 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^4 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^3) - I, sqrt(-5)/((1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^7 - (1/4*I*sqrt(5) + 1/4*sqrt(2*sqrt(5) + 10) - 1/4*I)^6 